In [3]:
import math
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib as mpl
import paste as pst
import scanpy as sc
import anndata
# from sklearn.decomposition import NMF
# import scanorama
import os
# import diopy
style.use('seaborn-dark')
mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14) 

##### WARNING: IF YOU ARE USING A VERSION OF PASTE THAT IS >v1.1.4 YOU MIGHT RUN INTO SOME SMALL ISSUES ############
########## ONE KNOWN ISSUE IS THAT THE INTERSECT FUNCTION IS NOT EXPORTED, an easy fix is to just do: 
from paste.helper import intersect

C:\Users\jinLab1\AppData\Local\Temp\ipykernel_10236\722246406.py:16: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  style.use('seaborn-dark')


In [12]:
path_to_output_dir = '../data/SCC/cached-results/'

#if not os.path.exists(path_to_output_dir):
#    os.makedirs(path_to_output_dir)

# Read in Data and Cache Adata

Saving it as .h5ad will make it faster to load the second time around.

In [6]:
def load_layer(patient, sample, metadata):
    """
    Return Layer object of Patient, Sample
    """
    layer_path = f"../data/SCC/scc_p{patient}_layer{sample}"
    layer = layer_path + ".tsv"
    coor_path = layer_path + "_coordinates.tsv"
    adata = anndata.read_csv(layer, delimiter="\t")

    # Data pre-processing
    coor = pd.read_csv(coor_path, sep="\t").iloc[:,:2]
    coor_index = []
    for pair in coor.values:
        coor_index.append('x'.join(str(e) for e in pair))
    coor.index = coor_index
    # The metadata, coordinates, and gene expression might have missing cells between them
    idx = intersect(coor_index, adata.obs.index)
    
    df = metadata[metadata['patient'] == patient]
    df = df[df['sample'] == sample]
    
    meta_idx = []
    for i in df.index:
        meta_idx.append(i.split('_')[1])
    idx = intersect(idx, meta_idx)
    
    adata = adata[idx, :]
    adata.obsm['spatial'] = np.array(coor.loc[idx, :])
    metadata_idx = ['P' + str(patient) + '_' + i + '_' + str(sample) for i in idx]
    adata.obs['original_clusters'] = [str(x) for x in list(metadata.loc[metadata_idx, 'SCT_snn_res.0.8'])]
    return adata

In [7]:
metadata_path =  "../data/SCC/ST_all_metadata.txt"
metadata = pd.read_csv(metadata_path, sep="\t", index_col=0)

adata_2_1 = load_layer(2, 1, metadata)
adata_2_2 = load_layer(2, 2, metadata)
adata_2_3 = load_layer(2, 3, metadata)
patient_2 = [adata_2_1, adata_2_2, adata_2_3]

adata_5_1 = load_layer(5, 1, metadata)
adata_5_2 = load_layer(5, 2, metadata)
adata_5_3 = load_layer(5, 3, metadata)
patient_5 = [adata_5_1, adata_5_2, adata_5_3]

adata_9_1 = load_layer(9, 1, metadata)
adata_9_2 = load_layer(9, 2, metadata)
adata_9_3 = load_layer(9, 3, metadata)
patient_9 = [adata_9_1, adata_9_2, adata_9_3]

adata_10_1 = load_layer(10, 1, metadata)
adata_10_2 = load_layer(10, 2, metadata)
adata_10_3 = load_layer(10, 3, metadata)
patient_10 = [adata_10_1, adata_10_2, adata_10_3]

patients = {
    "patient_2" : patient_2,
    "patient_5" : patient_5,
    "patient_9" : patient_9,
    "patient_10" : patient_10,
}

for p in patients.values():
    for adata in p:
        sc.pp.filter_genes(adata, min_cells = 15, inplace = True)
        sc.pp.filter_cells(adata, min_genes = 100, inplace = True)

In [13]:
H5ADs_dir = path_to_output_dir + 'H5ADs/'

if not os.path.exists(H5ADs_dir):
    os.makedirs(H5ADs_dir)
    
for k, p in patients.items():
    for i in range(len(p)):
        # print(p[i])
        # diopy.output.write_h5(p[i],file= (H5ADs_dir + k + '_slice_' + str(i) + '.h5'))
        p[i].write(H5ADs_dir + k + '_slice_' + str(i) + '.h5ad')

# Plot 3d

Read in data

In [14]:
path_to_h5ads = path_to_output_dir + 'H5ADs/'

patient_2 = []
patient_5 = []
patient_9 = []
patient_10 = []

patients = {
    "patient_2" : patient_2,
    "patient_5" : patient_5,
    "patient_9" : patient_9,
    "patient_10" : patient_10,
}

for k in patients.keys():
    for i in range(3):
        patients[k].append(sc.read_h5ad(path_to_h5ads + k + '_slice_' + str(i) + '.h5ad'))

Run pairwise align and calculate alignment based on pis

In [15]:
import plotly.express as px
import plotly.io as pio
import warnings
import plotly.graph_objects as go

In [19]:

for k, patient_n in patients.items():
# for k, patient_n in patients.items():
    pi12 = pst.pairwise_align(patient_n[0].copy(), patient_n[1].copy(), alpha = 0.1)
    pi23 = pst.pairwise_align(patient_n[1].copy(), patient_n[2].copy(), alpha = 0.1)
    pis = [pi12, pi23]
    new_slices = pst.stack_slices_pairwise(patient_n, pis)
    
    sub_dir = H5ADs_dir+k+"/"
    if not os.path.exists(sub_dir):
        os.makedirs(sub_dir)
        
    for i,pi in enumerate(pis):
        np.savetxt(fname=(sub_dir + "pi{0}{1}".format(i,i+1)+".csv"), X=pi, delimiter= ",")
    
    for i,L in enumerate(new_slices):
        L.write((sub_dir + "aligned_" + k + '_slice_' + str(i) + '.h5ad'))
        
    # warnings.filterwarnings("ignore")
    # ## comment out the following line and uncomment out the line below the next if you use jupyterlab instead jupyter notebooks.
    # pio.renderers.default='notebook'
    # #pio.renderers.default='jupyterlab' 
    # slice_colors = ['#e41a1c','#377eb8','#4daf4a','#984ea3']
    # # scale the distance between layers
    # z_scale = 10

    # df = pd.DataFrame(columns=['x','y','z','layer'])

    # clusters = []
    # for i,L in enumerate(new_slices):
    #     for x,y in L.obsm['spatial']:
    #         df = df._append({'x': x,'y':y,'z':i*z_scale,'slice':str(i)}, ignore_index=True)

    #     clusters.append(np.array(L.obs['original_clusters']))
    # clusters = np.concatenate(clusters)
    # df['clusters'] = clusters
    # fig = px.scatter_3d(df, x='x', y='y', z='z',color='clusters', title = k)
    # # fig = go.Figure(data=go.Scatter3d())
    # fig.update_layout(scene_aspectmode='data')
    # fig.show()
    

Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.
Using selected backend cpu. If you want to use gpu, set use_gpu = True.


In [15]:
pi12

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.50150150e-03, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.45261541e-03, 4.88860954e-05],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.50150150e-03]])

In [30]:
new_slices[0].obsm["spatial"]

array([[-20.1966967 ,   9.77027027],
       [-19.1966967 ,  -5.22972973],
       [-19.1966967 ,  -3.22972973],
       ...,
       [ 21.8033033 ,   9.77027027],
       [ 22.8033033 ,   6.77027027],
       [ 22.8033033 ,   8.77027027]])

In [ ]:
import plotly.express as px
df = px.data.gapminder().query("continent=='Europe'")
fig = px.line_3d(df, x="gdpPercap", y="pop", z="year", color='country')
fig.show()

In [2]:
print(pi12)

NameError: name 'pi12' is not defined

1. 查看seurat（R中的anndata）如何存储空间转录组多切片数据
2. R语言如何画3d的plotly图像
3. 看完文献2